# Phase 2: Create and Seed the Database

**Objective:** Bring our database to life by creating the physical `mining_knowledge.db` file and populating it with structured, validated sample data.

**Workflow:**
1.  **Define and test** the CRUD helper functions here in the notebook.
2.  **Define** sample data for our models.
3.  **Use** the helpers and data to seed the database.
4.  **Finally, copy** the finalized CRUD functions into a new `helper_crud.py` file.

---
## ✅ Part 1: Database and Session Setup

First, we'll set up the connection to our SQLite database and create the tables from our models.

### TODO 1.1: Imports


In [1]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker
import os
from dotenv import load_dotenv
from models import (    
    Base,
    SD_Objective,
    SDG_Goal,
    SDG_Target,
    SDG_Indicator,
    Practice,
    Stakeholder_Group,
    Stakeholder,
    Concern,
    PracticeToTargetLink,
    StakeholderToConcernLink,
    ConcernToTargetLink,
    SDObjectiveToSDGLink)


### TODO 1.2: Define Database URL and Create Engine

**Your Task:** Set the database URL and create the SQLAlchemy engine.

In [2]:
# This function will load variables from a .env file into your environment
load_dotenv()

# Retrieve the database URL from the environment.
# os.getenv() returns None if the variable is not found.
DATABASE_URL = os.getenv("DATABASE_URL")

# Check if the DATABASE_URL was loaded correctly
if not DATABASE_URL:
    raise ValueError("No DATABASE_URL found in the .env file or environment variables.")

# The 'echo=True' argument is very useful for development.
# It makes SQLAlchemy log all the SQL statements it generates.
engine = create_engine(DATABASE_URL, echo=True)

print(f"Engine created successfully using the URL from the .env file.")

Engine created successfully using the URL from the .env file.


The advantages of using the `.env` file to define DATABASE_URL:
- **Security**: You can add your .env file to .gitignore. This ensures that your database credentials or file paths are never accidentally committed to your public GitHub repository.

- **Flexibility**: You can have different .env files for different environments (development, testing, production) without ever changing your Python code.

- **Collaboration**: Your teammates can use their own local database configurations by simply creating their own .env file, making the project easier to set up for everyone.

### TODO 1.3: Create Database Tables

**Your Task:** Execute this cell to create the `mining_knowledge.db` file and all the tables within it. **Run this cell only once.** If you run it again after seeding data, it won't delete the data, but it's good practice to think of this as a one-time setup step.

In [3]:
# This command connects to the database and creates all tables
# that inherit from our 'Base' object in models.py
Base.metadata.create_all(bind=engine)

print("Database 'mining_knowledge.db' and its tables have been created or verified successfully.")

2025-07-15 14:24:01,337 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-15 14:24:01,338 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sd_objectives")
2025-07-15 14:24:01,339 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-15 14:24:01,342 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("sd_objectives")
2025-07-15 14:24:01,343 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-15 14:24:01,344 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("sdg_goal")
2025-07-15 14:24:01,344 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-15 14:24:01,345 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("sdg_goal")
2025-07-15 14:24:01,346 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-15 14:24:01,346 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("practice_action")
2025-07-15 14:24:01,347 INFO sqlalchemy.engine.Engine [raw sql] ()
2025-07-15 14:24:01,348 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("practice_action")
2025-07-15 14:24:01,349 INFO sqlalch

---
## 🛠️ Part 2: CRUD Helper Functions (for `helper_crud.py`)

Now, we will write the helper functions that will simplify our interactions with the database. Develop and test them in the cell below. Once they are working correctly, you can copy the entire content of the cell into a new `helper_crud.py` file.

### TODO 2.1 - 2.3: Create CRUD Helpers

**Your Task:** Write the Python code for the following functions:
1.  **Imports**: Import `Session`, all ORM models from `models`, and all `Create` schemas from `valid_schemas`.
2.  **`get_or_create(db: Session, model, **kwargs)`**: A generic function to prevent creating duplicate entries.
3.  **Specific `create_` functions**: For each model, write a function that takes a Pydantic `Create` schema and uses `get_or_create` to add it to the DB. For example: `create_practice(db: Session, practice: PracticeCreate)`.

In [4]:

# 1. Imports
from sqlalchemy.orm import Session
# It's good practice to import the modules themselves to maintain namespace clarity.
import models
import valid_schemas
import helper_crud as helper



✅ CRUD Helper functions defined and ready for use.


---
## 📝 Part 3: Prepare Sample Data

Here, we'll define the sample data we want to insert into our database. The data should be structured as a list of dictionaries, where each dictionary matches a Pydantic `Create` schema.

### TODO 3.1 & 3.2: Define Sample Data for Nodes and Links

**Your Task:** Create lists of dictionaries containing your sample data. I've provided a small example for `SD_Objective` and `Practice` to get you started.

In [5]:

# 1. Imports
from sqlalchemy.orm import Session
# It's good practice to import the modules themselves to maintain namespace clarity.
import models
import valid_schemas
import helper_crud as helper

print("CRUD Helper functions defined and ready for use.")

# TODO: Finish this part in Excel.
# SAMPLE_DATA AS FOLLOWS:

from models import LevelEnum

# --- Node Data ---

sd_objectives_data = [
    {"id": "Env", "description": "Environmental objectives related to minimizing negative impacts."},
    {"id": "Soc", "description": "Social objectives focused on community well-being and equity."},
    {"id": "Econ", "description": "Economic objectives focused on financial viability and benefit sharing."}
]

sdg_goals_data = [
    {"id": "SDG6", "name": "Clean Water and Sanitation", "parent_objective_id": "Env"},
    {"id": "SDG8", "name": "Decent Work and Economic Growth", "parent_objective_id": "Soc"},
    {"id": "SDG12", "name": "Responsible Consumption and Production", "parent_objective_id": "Econ"}
]

sdg_targets_data = [
    {"id": "6.3", "short_name": "Improve water quality", "description": "By 2030, improve water quality by reducing pollution, eliminating dumping and minimizing release of hazardous chemicals and materials...", "parent_goal_id": "SDG6"},
    {"id": "8.2", "short_name": "Improve economic productivity", "description": "Achieve higher levels of economic productivity through diversification, technological upgrading and innovation...", "parent_goal_id": "SDG8"},
    {"id": "12.2", "short_name": "Sustainable management of natural resources", "description": "By 2030, achieve the sustainable management and efficient use of natural resources.", "parent_goal_id": "SDG12"}
]

sdg_indicators_data = [
    {"id": "6.3.1", "description": "Proportion of domestic and industrial wastewater flows safely treated", "code": "C060301", "parent_target_id": "6.3"},
    {"id": "12.2.1", "description": "Material footprint, material footprint per capita, and material footprint per GDP", "code": "C120201", "parent_target_id": "12.2"}
]

practice_actions_data = [
    {"name": "Install water filtration systems", "description": "Deploy advanced filtration and purification technology at discharge points."},
    {"name": "Implement water recycling protocols", "description": "Establish closed-loop systems to reuse water in processing."},
    {"name": "Conduct regular water quality monitoring", "description": "Set up automated sensors and manual sampling schedules."}
]

practices_data = [
    {
        "id": "p1_water_management", "name": "Advanced Water Management", "category": "Environmental",
        "description": "A comprehensive strategy to minimize water use and prevent contamination.", "remark": "Requires significant upfront investment.",
        "evidence_source": "ISO 14001 Guidelines", "capital_intensity": LevelEnum.HIGH, "technical_complexity": LevelEnum.MEDIUM,
        "operational_disruption": LevelEnum.LOW, "long_term_liability": False
    },
    {
        "id": "p2_resource_efficiency", "name": "Resource Efficiency Program", "category": "Operational",
        "description": "A program to optimize the use of raw materials and energy.", "remark": "Focuses on reducing waste at the source.",
        "evidence_source": "Best Practice in Mining, 2022", "capital_intensity": LevelEnum.MEDIUM, "technical_complexity": LevelEnum.MEDIUM,
        "operational_disruption": LevelEnum.MEDIUM, "long_term_liability": False
    }
]

stakeholder_groups_data = [
    {"id": "shg1_community", "name": "Local Communities", "evidence": "Town hall meeting minutes, 2023-Q4"},
    {"id": "shg2_regulators", "name": "Government & Regulators", "evidence": "National Mining Act, Section 5.b"}
]

stakeholders_data = [
    {"id": "sh1_local_farmers", "name": "Downstream Farmers Association", "category_id": "shg1_community", "definition": "Agricultural producers located within 10km downstream of the mine site."},
    {"id": "sh2_epa", "name": "Environmental Protection Agency", "category_id": "shg2_regulators", "definition": "The national body responsible for enforcing environmental laws."}
]

concerns_data = [
    {"id": "con1_water_quality", "name": "Water Quality", "description": "Concern about the quality of river water used for irrigation and livestock.", "evidence": "Farmer's petition, Jan 2024"},
    {"id": "con2_compliance", "name": "Regulatory Compliance", "description": "Ensuring the mine operates within legally mandated environmental limits.", "evidence": "EPA Mandate #789-A"}
]


# --- Link Data ---

practice_to_action_links_data = [
    {"practice_id": "p1_water_management", "action_id": 1, "evidence": "Included in the initial design of the water management plan."},
    {"practice_id": "p1_water_management", "action_id": 2, "evidence": "Operational procedure manual, section 4.1"},
    {"practice_id": "p1_water_management", "action_id": 3, "evidence": "Environmental compliance commitment."}
]

practice_to_target_links_data = [
    {"practice_id": "p1_water_management", "target_id": "6.3", "relevance_weight": LevelEnum.HIGH, "is_direct": True, "evidence": "Technical report on water treatment efficacy."},
    {"practice_id": "p2_resource_efficiency", "target_id": "12.2", "relevance_weight": LevelEnum.HIGH, "is_direct": True, "evidence": "Waste reduction audit results."},
    {"practice_id": "p2_resource_efficiency", "target_id": "8.2", "relevance_weight": LevelEnum.LOW_MEDIUM, "is_direct": False, "evidence": "Economic analysis showing cost savings from efficiency."}
]

stakeholder_to_concern_links_data = [
    {"stakeholder_id": "sh1_local_farmers", "concern_id": "con1_water_quality", "priority_weight": LevelEnum.HIGH, "evidence": "Reported crop failures attributed to poor water quality."},
    {"stakeholder_id": "sh2_epa", "concern_id": "con1_water_quality", "priority_weight": LevelEnum.MEDIUM, "evidence": "Public complaints trigger official review."},
    {"stakeholder_id": "sh2_epa", "concern_id": "con2_compliance", "priority_weight": LevelEnum.HIGH, "evidence": "Core mandate of the agency."}
]

concern_to_target_links_data = [
    {"concern_id": "con1_water_quality", "target_id": "6.3", "evidence": "The concern directly maps to the goal of improving water quality."},
    {"concern_id": "con2_compliance", "target_id": "12.2", "evidence": "Compliance often involves resource management regulations."}
]

sd_objective_to_sdg_links_data = [
    {"sd_objective_id": "Env", "sdg_goal_id": "SDG6", "weight": LevelEnum.HIGH, "comment": "Water is a primary environmental vector."},
    {"sd_objective_id": "Soc", "sdg_goal_id": "SDG8", "weight": LevelEnum.HIGH, "comment": "Employment is a primary social vector."}
]

print("Sample data lists have been defined.")

CRUD Helper functions defined and ready for use.
Sample data lists have been defined.


---
## 🌱 Part 4: Seed the Database

This is the final step where we put everything together. We'll create a database session, loop through our sample data, validate it with Pydantic, use our CRUD helpers to add it to the session, and finally commit all changes to the database.

### TODO 4.1 - 4.4: Seeding Script

**Your Task:** Write the script to seed the database. Remember the correct order: **seed nodes first, then seed links** to ensure relational integrity.

In [6]:
# Part 4: Seeding Script

# Create a Session class to interact with the database
SessionLocal = sessionmaker(autocommit=False, autoflush=False, bind=engine)

# Create a new session
db = SessionLocal()

# try:
print("--- Seeding Node Data ---")

# Define a mapping that includes the correct argument name for each CRUD function
node_seeding_map = {
    # 'Name': (data_list, pydantic_schema, crud_function, argument_name)
    "SD Objectives": (sd_objectives_data, valid_schemas.SD_ObjectiveCreate, helper.create_sd_objective, "objective"),
    "SDG Goals": (sdg_goals_data, valid_schemas.SDG_GoalCreate, helper.create_sdg_goal, "goal"),
    "SDG Targets": (sdg_targets_data, valid_schemas.SDG_TargetCreate, helper.create_sdg_target, "target"),
    "SDG Indicators": (sdg_indicators_data, valid_schemas.SDG_IndicatorCreate, helper.create_sdg_indicator, "indicator"),
    "Practice Actions": (practice_actions_data, valid_schemas.PracticeActionCreate, helper.create_practice_action, "action"),
    "Practices": (practices_data, valid_schemas.PracticeCreate, helper.create_practice, "practice"),
    "Stakeholder Groups": (stakeholder_groups_data, valid_schemas.Stakeholder_GroupCreate, helper.create_stakeholder_group, "group"),
    "Stakeholders": (stakeholders_data, valid_schemas.StakeholderCreate, helper.create_stakeholder, "stakeholder"),
    "Concerns": (concerns_data, valid_schemas.ConcernCreate, helper.create_concern, "concern"),
}

# The corrected loop now uses the explicitly defined argument name
for name, (data_list, pydantic_schema, crud_function, arg_name) in node_seeding_map.items():
    for data_item in data_list:
        pydantic_obj = pydantic_schema(**data_item)
        crud_function(db=db, **{arg_name: pydantic_obj})
    print(f"{name} seeded.")

# Commit the nodes first to ensure they have IDs before creating links
db.commit()
print("\n--- Node data committed ---\n")

print("--- Seeding Link Data ---")

# The link seeding part was already correct as it used a consistent 'link' argument
link_seeding_map = {
    "Practice-to-Action Links": (practice_to_action_links_data, valid_schemas.PracticeToActionLinkCreate, helper.create_practice_to_action_link),
    "Practice-to-Target Links": (practice_to_target_links_data, valid_schemas.PracticeToTargetLinkCreate, helper.create_practice_to_target_link),
    "Stakeholder-to-Concern Links": (stakeholder_to_concern_links_data, valid_schemas.StakeholderToConcernLinkCreate, helper.create_stakeholder_to_concern_link),
    "Concern-to-Target Links": (concern_to_target_links_data, valid_schemas.ConcernToTargetLinkCreate, helper.create_concern_to_target_link),
    "SDObjective-to-SDG Links": (sd_objective_to_sdg_links_data, valid_schemas.SDObjectiveToSDGLinkCreate, helper.create_sd_objective_to_sdg_link),
}

for name, (data_list, pydantic_schema, crud_function) in link_seeding_map.items():
    for link_data in data_list:
        pydantic_obj = pydantic_schema(**link_data)
        crud_function(db=db, link=pydantic_obj)
    print(f"{name} seeded.")
    
# Final commit for the links
db.commit()
print("\n--- Link data committed ---")

print("\n✅ Database seeding completed successfully!")

# except Exception as e:
#     print(f"❌ An error occurred: {e}")
#     db.rollback() # Roll back the transaction on error

# finally:
#     db.close() # Always close the session
#     print("\nDatabase session closed.")

--- Seeding Node Data ---
2025-07-15 14:24:05,856 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2025-07-15 14:24:05,858 INFO sqlalchemy.engine.Engine SELECT sd_objectives.id AS sd_objectives_id, sd_objectives.description AS sd_objectives_description 
FROM sd_objectives 
WHERE sd_objectives.id = ? AND sd_objectives.description = ?
 LIMIT ? OFFSET ?
2025-07-15 14:24:05,859 INFO sqlalchemy.engine.Engine [generated in 0.00096s] ('Env', 'Environmental objectives related to minimizing negative impacts.', 1, 0)
2025-07-15 14:24:05,861 INFO sqlalchemy.engine.Engine SELECT sd_objectives.id AS sd_objectives_id, sd_objectives.description AS sd_objectives_description 
FROM sd_objectives 
WHERE sd_objectives.id = ? AND sd_objectives.description = ?
 LIMIT ? OFFSET ?
2025-07-15 14:24:05,861 INFO sqlalchemy.engine.Engine [cached since 0.003301s ago] ('Soc', 'Social objectives focused on community well-being and equity.', 1, 0)
2025-07-15 14:24:05,863 INFO sqlalchemy.engine.Engine SELECT sd_objective

2025-07-15 14:24:05,868 INFO sqlalchemy.engine.Engine SELECT sdg_goal.id AS sdg_goal_id, sdg_goal.name AS sdg_goal_name, sdg_goal.parent_objective_id AS sdg_goal_parent_objective_id 
FROM sdg_goal 
WHERE sdg_goal.id = ? AND sdg_goal.name = ? AND sdg_goal.parent_objective_id = ?
 LIMIT ? OFFSET ?
2025-07-15 14:24:05,868 INFO sqlalchemy.engine.Engine [cached since 0.002754s ago] ('SDG8', 'Decent Work and Economic Growth', 'Soc', 1, 0)
2025-07-15 14:24:05,870 INFO sqlalchemy.engine.Engine SELECT sdg_goal.id AS sdg_goal_id, sdg_goal.name AS sdg_goal_name, sdg_goal.parent_objective_id AS sdg_goal_parent_objective_id 
FROM sdg_goal 
WHERE sdg_goal.id = ? AND sdg_goal.name = ? AND sdg_goal.parent_objective_id = ?
 LIMIT ? OFFSET ?
2025-07-15 14:24:05,872 INFO sqlalchemy.engine.Engine [cached since 0.005952s ago] ('SDG12', 'Responsible Consumption and Production', 'Econ', 1, 0)
SDG Goals seeded.
2025-07-15 14:24:05,874 INFO sqlalchemy.engine.Engine SELECT sdg_target.id AS sdg_target_id, sdg_ta

In [7]:
db.close()
print("Session closed.")

Session closed.
